In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("ALPHAVANTAGE_API_KEY")
if not API_KEY:
    raise RuntimeError("Set ALPHAVANTAGE_API_KEY environment variable before running this cell")


def fetch_insider_transactions(symbol: str):
    params = {"function": "INSIDER_TRANSACTIONS", "symbol": symbol, "apikey": API_KEY}
    url = "https://www.alphavantage.co/query"
    resp = requests.get(url, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    return data


# Example: Fetch and save NVDA insider transactions
nvda_insiders = fetch_insider_transactions("NVDA")
nvda_insiders["data"][:5]

[{'transaction_date': '2025-10-15',
  'ticker': 'NVDA',
  'executive': 'HUANG, JEN HSUN',
  'executive_title': 'Director, President and CEO',
  'security_type': 'Common Stock',
  'acquisition_or_disposal': 'D',
  'shares': '6510.0',
  'share_price': '180.8338'},
 {'transaction_date': '2025-10-15',
  'ticker': 'NVDA',
  'executive': 'HUANG, JEN HSUN',
  'executive_title': 'Director, President and CEO',
  'security_type': 'Common Stock',
  'acquisition_or_disposal': 'D',
  'shares': '22944.0',
  'share_price': '180.09'},
 {'transaction_date': '2025-10-15',
  'ticker': 'NVDA',
  'executive': 'HUANG, JEN HSUN',
  'executive_title': 'Director, President and CEO',
  'security_type': 'Common Stock',
  'acquisition_or_disposal': 'D',
  'shares': '8768.0',
  'share_price': '179.1531'},
 {'transaction_date': '2025-10-15',
  'ticker': 'NVDA',
  'executive': 'HUANG, JEN HSUN',
  'executive_title': 'Director, President and CEO',
  'security_type': 'Common Stock',
  'acquisition_or_disposal': 'D',
 

In [2]:
import pandas as pd


df = pd.DataFrame(nvda_insiders["data"])
df

,transaction_date,ticker,executive,executive_title,security_type,acquisition_or_disposal,shares,share_price
0,2025-10-15,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,6510.0,180.8338
1,2025-10-15,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,22944.0,180.09
2,2025-10-15,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,8768.0,179.1531
3,2025-10-15,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,2917.0,178.1301
4,2025-10-15,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,90.0,184.6322
...,...,...,...,...,...,...,...,...
6631,2004-04-13,NVDA,"MA, DI","Vice President, Operations",Employee Stock Option (right to buy),A,60000.0,0.0
6632,2004-04-13,NVDA,"SHANNON, DAVID M","VP, General Counsel",Employee Stock Option (right to buy),A,60000.0,0.0
6633,2004-04-13,NVDA,"VIVOLI, DANIEL","Vice President, Marketing",Employee Stock Option (right to buy),A,60000.0,0.0
6634,2004-02-27,NVDA,"BURKETT, MARVIN D",Chief Financial Officer,Common Stock,A,1861.0,8.06


In [4]:
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
date_range_mask = (df["transaction_date"] >= "2020-01-01") & (df["transaction_date"] <= "2025-06-30")
filter_df = df[date_range_mask]
filter_df = filter_df.sort_values(by="transaction_date").reset_index(drop=True)
filter_df

,transaction_date,ticker,executive,executive_title,security_type,acquisition_or_disposal,shares,share_price
0,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,200.0,232.25
1,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,300.0,233.85
2,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,293.0,236.82
3,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,907.0,236.28
4,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,300.0,233.85
...,...,...,...,...,...,...,...,...
3069,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,20463.0,156.7071
3070,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,26519.0,157.3152
3071,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,3930.0,158.1053
3072,2025-06-30,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,19624.0,156.8135


In [5]:
filter_df.to_parquet("insiders/NVDA_insider_transactions_2020_2025.parquet", index=False)

In [6]:
test_load = pd.read_parquet("insiders/NVDA_insider_transactions_2020_2025.parquet")
test_load

,transaction_date,ticker,executive,executive_title,security_type,acquisition_or_disposal,shares,share_price
0,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,200.0,232.25
1,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,300.0,233.85
2,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,293.0,236.82
3,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,907.0,236.28
4,2020-01-06,NVDA,"KRESS, COLETTE",EVP & Chief Financial Officer,Common,D,300.0,233.85
...,...,...,...,...,...,...,...,...
3069,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,20463.0,156.7071
3070,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,26519.0,157.3152
3071,2025-06-30,NVDA,"SEAWELL, A BROOKE",Director,Common Stock,D,3930.0,158.1053
3072,2025-06-30,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,19624.0,156.8135
